In [2]:
import os
import time
import pickle
from datetime import datetime

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

from tqdm.notebook import tqdm

import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import ExponentialLR

from util_func import sMAPE, RMSE, MAE

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
def read_weather(data_root, force_reload=False):
    try:
        with open(data_root+"loaded_dataset.pk", 'rb') as f:
            df = pickle.load(f)
    except:
        force_reload = True
    
    if force_reload:
        df = pd.read_csv(data_root+"ACISHourlyData-20180101-20211231.csv")
        df.drop("Station Name", axis=1, inplace=True)
        df["DATETIME"] = df.apply(lambda x: datetime.strptime(str(x["DATE"])+str(x["HOUR_ENDING"]-1), "%Y%m%d%H"), axis=1)
        df.drop("DATE", axis=1, inplace=True)
        df = df[["SITE_ID", "RATE_CLASS", "DATETIME", "IS_DAYLIGHT_SAVING", "CONSUMPTION_KWH"]]
        df.sort_values(["SITE_ID", "DATETIME"], ascending=True, inplace=True)

        with open(data_root+"loaded_dataset.pk", 'wb') as f:
            pickle.dump(df, f)

    return df

def preprocess_weather():
    pass

In [4]:
data_root = "../Datasets/Time_Series_Datasets/Weather/Edmonton/"
df = pd.read_csv(data_root+"ACISHourlyData-20180101-20211231.csv")

In [8]:
df.head()

,Station Name,Date (Local Standard Time),Air Temp. Avg. (C),Humidity Avg. (%),Incoming Solar Rad. (W/m2),Precip. Accumulated (mm),Precip. (mm),Snow Depth (cm),Wind Speed 10 m Avg. (km/h)
0,Edmonton South Campus UA,01-January-2018 00:00,-26.3,67.3,0.0,0.0,0.0,-0.8,8.5
1,Edmonton South Campus UA,01-January-2018 01:00,-26.6,67.9,0.0,0.0,0.0,-0.8,8.3
2,Edmonton South Campus UA,01-January-2018 02:00,-27.0,67.2,0.0,0.0,0.0,-0.7,9.0
3,Edmonton South Campus UA,01-January-2018 03:00,-27.2,67.6,0.0,0.0,0.0,-0.8,8.3
4,Edmonton South Campus UA,01-January-2018 04:00,-27.5,67.9,0.0,0.0,0.0,-0.7,8.4


In [7]:
type(df.iloc[0]["Date (Local Standard Time)"])

str